<a href="https://colab.research.google.com/github/abdulrehman-faisal100/Nuclear-Medicince-CNN/blob/main/Cancer_Diagnosis_with_AI_on_FPGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
import numpy as np
import os
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import shutil
import random
import pandas as pd
import keras
from PIL import Image
from numpy import asarray
import tensorflow as tf
from sklearn.metrics import f1_score
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import img_to_array, load_img
import zipfile

In [ ]:
heights = []
widths = []
# Define the path to the parent folder
parent_folder_path = os.path.join('/content/drive/MyDrive/chestLANT/train')

# Get a list of the folder names inside the parent folder
folder_names = [name for name in os.listdir(parent_folder_path) if os.path.isdir(os.path.join(parent_folder_path, name))]

# Print the folder names
for names in folder_names:
  fold_path=os.path.join(parent_folder_path,names)
  for filename in os.listdir(fold_path):
      if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
          img_path = os.path.join(fold_path, filename)
          img = Image.open(img_path)
          width, height = img.size
          heights.append(height)
          widths.append(width)

# Calculate the median height and width
median_height = sorted(heights)[len(heights)//2]
median_width = sorted(widths)[len(widths)//2]

# Print the results
print(f"Median height: {median_height}")
print(f"Median width: {median_width}")

Median height: 171
Median width: 66


In [ ]:
def train_test_separate(folderName: str):
  # Define the path to the parent folder
  parent_folder_path = os.path.join('/content/drive/MyDrive', folderName)
  output_dir = "/content/drive/MyDrive/" + str(folderName) + "_processed"
  if not os.path.exists(output_dir):
      os.makedirs(output_dir)
  # Get a list of the folder names inside the parent folder
  folder_names = [name for name in os.listdir(parent_folder_path) if os.path.isdir(os.path.join(parent_folder_path, name))]
  train_dir = os.path.join(output_dir, 'train')
  val_dir = os.path.join(output_dir, 'test')
  if not os.path.exists(train_dir):
      os.makedirs(train_dir)
  if not os.path.exists(val_dir):
      os.makedirs(val_dir)
  # Print the folder names
  for names in folder_names:
    data_dir=os.path.join(parent_folder_path,names)
    # set the ratio of the validation set
    val_ratio = 0.2
    img_filenames = os.listdir(data_dir)
    print(img_filenames)

    newtrain_dir = os.path.join(train_dir, names)
    newtest_dir = os.path.join(val_dir, names)
    if not os.path.exists(newtrain_dir):
        os.makedirs(newtrain_dir)
    if not os.path.exists(newtest_dir):
        os.makedirs(newtest_dir)
    # divide the data into training and validation sets
    train_filenames, val_filenames = train_test_split(img_filenames, test_size=val_ratio, random_state=42)

    # copy the training set to the output directory
    for filename in train_filenames:
        src_path = os.path.join(data_dir, filename)
        dst_path = os.path.join(newtrain_dir, filename)
        shutil.copy(src_path, dst_path)

    # copy the validation set to the output directory
    for filename in val_filenames:
        src_path = os.path.join(data_dir, filename)
        dst_path = os.path.join(newtest_dir, filename)
        shutil.copy(src_path, dst_path)

In [ ]:
train_test_separate("Agricultural-crops")

['image (12).jpeg', 'image (10).jpg', 'image (11).jpeg', 'image (11).jpg', 'image (10).jpeg', 'image (1).png', 'image (1).jpg', 'image (1).jpeg', 'image (8).jpg', 'image (9).jpeg', 'image (8).jpeg', 'image (7).jpg', 'image (7).jpeg', 'image (6).jpg', 'image (5).jpeg', 'image (5).jpg', 'image (3).jpeg', 'image (6).jpeg', 'image (3).jpg', 'image (20).jpg', 'image (2).jpeg', 'image (4).jpg', 'image (4).jpeg', 'image (19).jpg', 'image (15).jpg', 'image (18).jpg', 'image (14).jpg', 'image (13).jpg', 'image (12).jpg']
['image (1).jpeg', 'image (12).jpg', 'image (15).jpg', 'image (14).jpg', 'image (11).jpg', 'image (13).jpg', 'image (8).jpeg', 'image (8).jpg', 'image (9).jpg', 'image (6).jpg', 'image (6).jpeg', 'image (7).jpg', 'image (7).jpeg', 'image (5).jpg', 'image (4).jpg', 'image (26).jpg', 'image (4).jpeg', 'image (25).jpg', 'image (3).jpg', 'image (5).jpeg', 'image (22).jpg', 'image (21).jpg', 'image (2).jpg', 'image (2).jpeg', 'image (16).jpg']
['image7.jpeg', 'images14.jpg', 'image5

In [ ]:
def preprocess(folderName: str):
  heights = []
  widths=[]
  folders = ["train", "val"]
  for folder in folders:
    # Define the path to the parent folder
    parent_folder_path = "/content/drive/MyDrive/" + str(folderName) + "/" + folder
    # Get a list of the folder names inside the parent folder
    folder_names = [name for name in os.listdir(parent_folder_path) if os.path.isdir(os.path.join(parent_folder_path, name))]
    for names in folder_names:
      # Print the folder names
      img_dir=os.path.join(parent_folder_path,names)
      for img1 in os.listdir(img_dir):
        img = cv2.imread(str(img_dir)+"/"+str(img1))
        desired_size = (75,171)

        resized = cv2.resize(img, desired_size)
        print(resized.shape)
        output_dir= '/content/drive/MyDrive/chestLANT_processed'
        # Save the sharpened image
        output_dir = os.path.join(output_dir, folder,names)
        if not os.path.exists(output_dir):
          os.makedirs(output_dir)
        output_dir = os.path.join(output_dir, img1)
        cv2.imwrite(output_dir, resized)
        print(output_dir)

In [ ]:
preprocess("chestLANT")

In [2]:
# Define constants
img_height, img_width = 171, 75
batch_size = 32
num_classes = 2

# Define data generators
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, rotation_range=20, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/chestLANT_processed/train',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

test_generator = test_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/chestLANT_processed/val',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
# Load pre-trained Inceptionv3 model
inceptionv3_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze all layers except the last few layers for fine-tuning
for layer in inceptionv3_model.layers[:-10]:
    layer.trainable = False

# Add custom layers for classification
x = inceptionv3_model.output
#x = tf.keras.layers.Conv2D(3, (1,1), activation='relu')(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024,activation=tf.nn.sigmoid)(x)
#x = tf.keras.layers.Dense(512,activation=tf.nn.sigmoid)(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(2,activation=tf.nn.sigmoid)(x)
predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

# Create the final model
model = tf.keras.models.Model(inputs=inceptionv3_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# Train the model
model.fit(train_generator, epochs=1, steps_per_epoch=train_generator.samples//batch_size, validation_data=test_generator, validation_steps=test_generator.samples//batch_size)
model.save('/content/drive/MyDrive/cancer_classifier_model.h5')
model.summary()

Found 2339 images belonging to 2 classes.
Found 586 images belonging to 2 classes.


73/73 [==============================] - 559s 7s/step - loss: 0.2815 - accuracy: 0.9293 - precision: 0.9293 - recall: 0.9293 - val_loss: 0.2084 - val_accuracy: 0.9601 - val_precision: 0.9601 - val_recall: 0.9601
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 171, 75, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 85, 37, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 85, 37, 32)  96          ['conv2d[0][0]'

In [2]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/chestLANT_processed/val',
    batch_size=32,
    class_mode='categorical',
    shuffle=False)
model = keras.models.load_model('/content/drive/MyDrive/cancer_classifier_model.h5')
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
# Get the predictions for the validation data
val_preds = model.predict(test_generator)
# Convert the predictions to labels
val_labels = np.argmax(val_preds, axis=1)
# Get the ground truth labels
val_ground_truth = test_generator.classes
# Generate a classification report and confusion matrix
print(classification_report(val_ground_truth, val_labels))
cm = confusion_matrix(val_ground_truth, np.argmax(val_preds, axis=1))
print(cm)

Found 586 images belonging to 2 classes.
19/19 [==============================] - 114s 6s/step
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       553
           1       0.00      0.00      0.00        33

    accuracy                           0.94       586
   macro avg       0.47      0.50      0.49       586
weighted avg       0.89      0.94      0.92       586

[[553   0]
 [ 33   0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [3]:
flatten_index = None
for i, layer in enumerate(model.layers):
    if isinstance(layer, keras.layers.Flatten):
        flatten_index = i
        break
print(f"Flatten index: {flatten_index}")

Flatten index: 312


In [11]:
def makeTestData(folderName: str):
  model = keras.models.load_model('/content/drive/MyDrive/cancer_classifier_model.h5')
  # Get a reference to the mixed10 layer
  mixed10_layer = model.get_layer('mixed10')
  # Create a new model that only includes up to the mixed10 layer
  mixed10_model = tf.keras.models.Model(inputs=model.input, outputs=mixed10_layer.output)
  output_dir = "/content/drive/MyDrive/TestData/"
  classes = ["0","1"]  
  folders = ["train","val"]  
  for i in folders:
    for j in classes:
      dir_path = os.path.join("/content/drive/MyDrive/",folderName,i,j)
      for filename in os.listdir(dir_path):
        # Load your image
        img = cv2.imread(os.path.join(dir_path, filename))
        numpydata = asarray(img)
        numpydata = np.expand_dims(numpydata, axis=0)
        print(numpydata.shape)
        # Get the output of the mixed10 layer for the image
        mixed10_output = mixed10_model.predict(numpydata)
        mixed10_output = np.squeeze(mixed10_output, axis=0)
        # Rescale the output to values between 0 and 255
        mixed10_output = (mixed10_output - mixed10_output.min()) / (mixed10_output.max() - mixed10_output.min())*255
        print(mixed10_output.shape)
        array =mixed10_output
          
        print(array.shape)
          
        array = np.reshape(array, (128,64))
        # show the shape of the array
        print(array.shape)
          
        data = Image.fromarray(array)
        data = data.convert('RGB')
        width = data.width
        height = data.height
        channel = data.mode
        # display width and height
        print("The height of the image is: ", height)
        print("The width of the image is: ", width)
        print("The channel of the image is: ", channel)
        # saving the final output 
        # as a PNG file
        output = os.path.join(output_dir,i,j)
        if not os.path.exists(output):
          os.makedirs(output)
        data.save(os.path.join(output,filename))

In [ ]:
makeTestData("chestLANT_processed")

In [4]:
from PIL import Image
import os
def imageTOnumpy(folderName: str):
  # Set the directory path
  classes = ["0","1"]  
  folders = ["train", "val"]
  x_train = []
  y_train = []
  x_test = []
  y_test = []
  for i in folders:
    for j in classes:
      dir_path = os.path.join("/content/drive/MyDrive/",folderName,i,j)
      # Loop through all files in the directory
      for filename in os.listdir(dir_path):
          # Check if the file is an image
          if filename.endswith('.jpg'):
              # Load the image using Pillow
              img = Image.open(os.path.join(dir_path, filename))
              # Convert the image to a numpy array
              img_arr = np.array(img)
              if i == "train":
                # Append the image array to the list
                x_train.append(img_arr)
                y_train.append(int(j))
              elif i == "val":
                x_test.append(img_arr)
                y_test.append(int(j))
  # Convert the list of image arrays to a numpy array
  x_train = np.array(x_train)
  x_test = np.array(x_test)
  y_train = np.array(y_train)
  y_test = np.array(y_test)
  return x_train, y_train, x_test, y_test

In [5]:
x_train, y_train, x_test, y_test = imageTOnumpy("TestData")

In [26]:
print(y_train.shape)

(2339,)


In [6]:
import tensorflow as tf
import json

x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024,activation=tf.nn.sigmoid))
model.add(tf.keras.layers.Dense(100,activation=tf.nn.sigmoid))
model.add(tf.keras.layers.Dense(2,activation=tf.nn.sigmoid))
model.add(tf.keras.layers.Dense(2,activation=tf.nn.softmax))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(x_train,y_train,epochs=2)
(val_loss,val_accuracy) = model.evaluate(x_test,y_test)
#--------From here onwards the weights & biases formatting is done-------# 
weightList = []
biasList = []
for i in range(1,len(model.layers)):
    weights = model.layers[i].get_weights()[0] #list of weights
    weightList.append((weights.T).tolist())
    bias = [[float(b)] for b in model.layers[i].get_weights()[1]] #list of biases 
    biasList.append(bias)

data = {"weights": weightList,"biases":biasList}
f = open('weightsandbiases.txt', "w")
json.dump(data, f)
f.close()

Epoch 1/2
74/74 [==============================] - 37s 469ms/step - loss: 0.2606 - accuracy: 0.9295
Epoch 2/2
19/19 [==============================] - 2s 75ms/step - loss: 0.2203 - accuracy: 0.9437


In [14]:
model.summary()
print(x_test.shape)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 24576)             0         
                                                                 
 dense (Dense)               (None, 1024)              25166848  
                                                                 
 dense_1 (Dense)             (None, 100)               102500    
                                                                 
 dense_2 (Dense)             (None, 2)                 202       
                                                                 
 dense_3 (Dense)             (None, 2)                 6         
                                                                 
Total params: 25,269,556
Trainable params: 25,269,556
Non-trainable params: 0
_________________________________________________________________
(586, 128, 64, 3)


In [ ]:
#model = keras.models.load_model('/content/drive/MyDrive/cancer_classifier_model.h5')
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
# Get the predictions for the validation data
val_preds = model.predict(x_test)
# Convert the predictions to labels
val_labels = np.argmax(val_preds, axis=1)
print(val_labels)
# Get the ground truth labels
val_ground_truth = y_test
print(val_ground_truth)
# Generate a classification report and confusion matrix
print(classification_report(val_ground_truth, val_labels))
cm = confusion_matrix(val_ground_truth, np.argmax(val_preds, axis=1))
print(cm)

In [21]:
import sys

outputPath = "/content/drive/MyDrive/TestDataFPGA/"
if not os.path.exists(outputPath):
  os.makedirs(outputPath)
headerFilePath = "./content/drive/MyDrive/TestDataFPGA/"
x_test = tf.keras.utils.normalize(x_test, axis=1)
import pickle
import gzip
import numpy as np

dataWidth = 8                  #specify the number of bits in test data
IntSize = 1 #Number of bits of integer portion including sign bit

try:
    testDataNum = int(sys.argv[1])
except:
    testDataNum = 3

def DtoB(num,dataWidth,fracBits):                        #funtion for converting into two's complement format
    if num >= 0:
        num = num * (2**fracBits)
        d = int(num)
    else:
        num = -num
        num = num * (2**fracBits)        #number of fractional bits
        num = int(num)
        if num == 0:
            d = 0
        else:
            d = 2**dataWidth - num
    return d
te_d = (x_test,y_test)
test_inputs = [np.reshape(x, (1, 24576)) for x in te_d[0]]
x = len(test_inputs[0][0])
d=dataWidth-IntSize
for i in range(len(test_inputs)):
    if i < 10:
        ext = "000"+str(i)
    elif i < 100:
        ext = "00"+str(i)
    elif i < 1000:
        ext = "0"+str(i)
    else:
        ext = str(i)
    fileName = 'test_data_'+ext+'.txt'
    f = open(outputPath+fileName,'w')
    for j in range(0,x):
        dInDec = DtoB(test_inputs[i][0][j],dataWidth,d)
        myData = bin(dInDec)[2:]
        f.write(myData+'\n')
    f.write(bin(DtoB((te_d[1][i]),dataWidth,0))[2:])
    f.close()